In [1]:
import pandas as pd
import os

In [2]:
# =========
# H1B
# =========

# Dictionary
year_to_df = {}

# Load all files
for year in range(2009, 2025):
    year_str = str(year)
    filepath = f"./data/h1b/{year_str}.csv"
    yearly_df = pd.read_csv(filepath, encoding="utf-16", sep="\t")
    year_to_df[year] = yearly_df

/var/folders/lq/03ylk8j930n85kgwjsdqd0ym0000gn/T/ipykernel_15724/3973809008.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  yearly_df = pd.read_csv(filepath, encoding="utf-16", sep="\t")


In [3]:
# Re-index
for year in range(2009, 2025):
    df = year_to_df[year]

    df["Industry (NAICS) Code"].fillna("Unknown", inplace=True)

    for col in ["Initial Approval", "Continuing Approval"]:
        df[col] = (
            df[col]
            .astype(str)  # Ensure it's a string for replacement
            .str.replace(",", "", regex=True)  # Remove commas
            .astype(float)  # Convert to float (handles NaNs)
            .fillna(0)  # Replace NaNs with 0
            .astype(int)  # Convert to integer
        )

    # Compute total approvals
    df["Total Approvals"] = df["Initial Approval"] + df["Continuing Approval"]

    # Ensure 'Total Approvals' is properly numeric
    df["Total Approvals"] = (
        pd.to_numeric(df["Total Approvals"], errors="coerce").fillna(0).astype(int)
    )

    # Re-index with (State, Industry) and sum approvals
    df_grouped = (
        df.groupby(["Petitioner State", "Industry (NAICS) Code"])["Total Approvals"]
        .sum()
        .reset_index()
    )

    # Set as multi-index
    df_grouped.set_index(["Petitioner State", "Industry (NAICS) Code"], inplace=True)
    df_grouped.to_csv(f"data/h1b_processed/{str(year)}.csv")

In [4]:
# =========
# Labor
# =========

# Dictionary
quarter_to_df = {}

# Load all files
for year in range(2003, 2021):
    year_str = str(year)
    filepath = f"./data/labor/{year_str}/"
    for month in {"jan", "apr", "jul", "oct"}:
        quarterly_filepath = filepath + month + ".xlsx"
        quarterly_df = pd.read_excel(quarterly_filepath)
        quarter_to_df[year_str + month] = quarterly_df

In [7]:
import collections

months = ["jan", "apr", "jul", "oct"]
new_labor_df = collections.defaultdict(int)

In [14]:
import re

def labor_helper_1(year, month, col1, col2, col3, col4, range1, range2, range3):
    cols = [col1, col2, col3, col4]
    for i, industry in enumerate(
        ["Total", "Natural Res. and Mining", "Construction", "Manufacturing"]
    ):
        for row in range1:
            state_raw = df.loc[row, "State"]

            state_trimmed = re.sub(r"\s*\.*$", "", state_raw.strip())
            k = tuple([year, month, state_trimmed, industry])
            new_labor_df[k] = df.loc[row, cols[i]]

    for i, industry in enumerate(
        ["Trade, Transport, Util.", "Info", "Finance", "Prof. Bus. Services"]
    ):
        # cols = [col1, "Unnamed: 12", col3, col4] # For 2005 jul
        # cols = [col1, "Unnamed: 13", col3, col4] # For 2005 oct to 2010 apr
        # cols = [ "Unnamed: 6", "Unnamed: 15", "Unnamed: 23", "Unnamed: 31"]  # For 2010 jul
        cols = [ col1, "Unnamed: 14", "Unnamed: 22", "Unnamed: 30"]  # For 2010 oct to 2011 jul
        for row in range2:
            state_raw = df.loc[row, "State"]
            state_trimmed = re.sub(r"\s*\.*$", "", state_raw.strip())
            k = tuple([year, month, state_trimmed, industry])
            if industry == "Info":
                print(df.loc[row, cols[i]])
            new_labor_df[k] = df.loc[row, cols[i]]

    for i, industry in enumerate(
        ["Education and health", "Leisure and hospitality", "Other", "Gov"]
    ):
        for row in range3:
            state_raw = df.loc[row, "State"]
            state_trimmed = re.sub(r"\s*\.*$", "", state_raw.strip())
            k = tuple([year, month, state_trimmed, industry])
            new_labor_df[k] = df.loc[row, cols[i]]


def labor_helper_2(year, month, col1, col2, col3, range1, range2, range3, range4):
    cols = [col1, col2, col3]
    for i, industry in enumerate(
        ["Total", "Natural Res. and Mining", "Construction"]
    ):
        for row in range1:
            state_raw = df.iloc[row, 0]
            state_trimmed = re.sub(r"\d+\s*|\.+", "", state_raw.strip()).strip()
            k = tuple([year, month, state_trimmed, industry])
            print(df.loc[row, cols[i]])
            new_labor_df[k] = df.loc[row, cols[i]]

    for i, industry in enumerate(
        ["Manufacturing", "Trade, Transport, Util.", "Info"]
    ):
        for row in range2:
            state_raw = df.iloc[row, 0]
            state_trimmed = re.sub(r"\d+\s*|\.+", "", state_raw.strip()).strip()
            k = tuple([year, month, state_trimmed, industry])
            print(df.loc[row, cols[i]])
            new_labor_df[k] = df.loc[row, cols[i]]

    for i, industry in enumerate(
        ["Finance", "Prof. Bus. Services", "Education and health"]
    ):
        for row in range3:
            state_raw = df.iloc[row, 0]
            state_trimmed = re.sub(r"\d+\s*|\.+", "", state_raw.strip()).strip()
            k = tuple([year, month, state_trimmed, industry])
            print(df.loc[row, cols[i]])
            new_labor_df[k] = df.loc[row, cols[i]]

    for i, industry in enumerate(
        ["Leisure and hospitality", "Other", "Gov"]
    ):
        for row in range4:
            state_raw = df.iloc[row, 0]
            state_trimmed = re.sub(r"\d+\s*|\.+", "", state_raw.strip()).strip()
            k = tuple([year, month, state_trimmed, industry])
            print(df.loc[row, cols[i]])
            new_labor_df[k] = df.loc[row, cols[i]]


# 2003 to 2004 (inclusive)
# for year in range(2003, 2005):
#     for i, month in enumerate(months):
#         if year == 2006 and month in {"jul", "oct"}:
#             continue
#         df = quarter_to_df[str(year) + month]
#         labor_helper_1(
#             year,
#             month,
#             "Unnamed: 4",
#             "Unnamed: 9",
#             "Unnamed: 13",
#             "Unnamed: 17",
#             range(2, 54),
#             range(57, 109),
#             range(112, 164),
#         )

# 2005 jan
# df = quarter_to_df["2005jan"]
# labor_helper_1(
#     2005,
#     "jan",
#     "Unnamed: 4",
#     "Unnamed: 9",
#     "Unnamed: 13",
#     "Unnamed: 17",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2005 apr
# df = quarter_to_df["2005apr"]
# labor_helper_1(
#     2005,
#     "apr",
#     "Unnamed: 4",
#     "Unnamed: 9",
#     "Unnamed: 13",
#     "Unnamed: 17",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2005 jul
# df = quarter_to_df["2005jul"]
# labor_helper_1(
#     2005,
#     "jul",
#     "Unnamed: 6",
#     "Unnamed: 13",
#     "Unnamed: 17",
#     "Unnamed: 21",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2005 oct
# df = quarter_to_df["2005oct"]
# labor_helper_1(
#     2005,
#     "oct",
#     "Unnamed: 6",
#     "Unnamed: 14",
#     "Unnamed: 18",
#     "Unnamed: 22",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2006 to 2010 apr (inclusive)
# for year in range(2006, 2011):
#     for i, month in enumerate(months):
#         if year == 2010 and month == "oct" or month == "jul":
#             continue
#         df = quarter_to_df[str(year) + month]
#         labor_helper_1(
#             year,
#             month,
#             "Unnamed: 6",
#             "Unnamed: 14",
#             "Unnamed: 18",
#             "Unnamed: 22",
#             range(2, 54),
#             range(57, 109),
#             range(112, 164),
#         )

# 2010 jul
# df = quarter_to_df["2010jul"]
# labor_helper_1(
#     year,
#     month,
#     "Unnamed: 7",
#     "Unnamed: 16",
#     "Unnamed: 24",
#     "Unnamed: 32",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2010 oct
# df = quarter_to_df["2010oct"]
# labor_helper_1(
#     year,
#     month,
#     "Unnamed: 6",
#     "Unnamed: 15",
#     "Unnamed: 23",
#     "Unnamed: 31",
#     range(2, 54),
#     range(57, 109),
#     range(112, 164),
# )

# 2011 jan to 2011 jul (inclusive)
# for i, month in enumerate(months):
#     if month == "oct":
#         continue
#     df = quarter_to_df["2011" + month]
#     labor_helper_1(
#         2011,
#         month,
#         "Unnamed: 6",
#         "Unnamed: 15",
#         "Unnamed: 23",
#         "Unnamed: 31",
#         range(2, 55),
#         range(58, 111),
#         range(114, 167),
#     )

# 2011 oct to 2020 oct (end)
for year in range(2011, 2021):
    for i, month in enumerate(months):
        if year == 2011 and month != "oct":
            continue
        df = quarter_to_df[str(year) + month]
        labor_helper_2(
            year,
            month,
            "Unnamed: 4",
            "Unnamed: 8",
            "Unnamed: 12",
            range(3, 56),
            range(60, 113),
            range(116, 169),
            range(172, 225)
        )

1880.7
327.1
2433.5
1180.5
14227.7
2261.2
1640.3
415.3
716.4
7261.9
3818.8
597
617.4
5743
2828.7
1504.8
1340.9
1811.8
1939.9
605.7
2547.7
3274.9
3987.6
2701.9
1104.4
2671
440.4
967.3
1130
636.6
3894.7
811.8
8727
3909
405.9
5166.6
1584.2
1645.5
5748.4
466.3
1832.5
410
2667.9
10681.9
1226.8
308.6
3681.6
2848.8
762.5
2779.4
292.5
916
42.2
12.7
16.8
11.1
10.9
28.6
28.8
0.6
–
–
5.7
9
–
4
9.6
6.8
2.2
9.3
23
55.7
2.8
–
1.1
8
6.6
9.1
4.4
8.3
–
12.8
0.9
1.5
20.7
5.7
5.7
17.5
12.2
49.5
7
34.1
0.2
4
–
–
256.8
11.7
0.8
10.5
6.1
33.1
3.3
28.3
–
–
81.9
17.5
118.6
49
582
113.7
53.1
19.7
11.1
327.2
134.4
28.7
32.6
218.8
129.1
68.2
56.6
67.1
125.4
26.4
143.1
114.3
139.8
95.9
47.5
105.8
24.7
44.2
56.8
21.6
135.8
41.4
325.7
174.3
28
187.4
74.2
74.4
230.8
16.8
76.4
22.7
113.2
598.1
70.7
14.9
184.5
146
35.7
94
24.7
29.6
2
237.7
9.5
151.8
154.6
1256.2
125.9
166.4
26.3
1.1
306.9
345.6
12.8
55.7
572.5
457.3
208.9
164.1
218.1
146.9
50.9
112.8
260
506.4
299.4
133.1
255.3
16.3
94.5
35.9
66.1
250.3
29.7
452.4
433

In [20]:
new_labor_df

defaultdict(int,
            {(2005, 'jan', 'Alabama', 'Total'): 1904.9,
             (2005, 'jan', 'Alaska', 'Total'): 288.6,
             (2005, 'jan', 'Arizona', 'Total'): 2408,
             (2005, 'jan', 'Arkansas', 'Total'): 1151.6,
             (2005, 'jan', 'California', 'Total'): 14507.4,
             (2005, 'jan', 'Colorado', 'Total'): 2174.8,
             (2005, 'jan', 'Connecticut', 'Total'): 1638.1,
             (2005, 'jan', 'Delaware', 'Total'): 419.8,
             (2005, 'jan', 'District of Columbia', 'Total'): 665.1,
             (2005, 'jan', 'Florida', 'Total'): 7610.2,
             (2005, 'jan', 'Georgia', 'Total'): 3853.5,
             (2005, 'jan', 'Hawaii', 'Total'): 583.9,
             (2005, 'jan', 'Idaho', 'Total'): 574.9,
             (2005, 'jan', 'Illinois', 'Total'): 5694.4,
             (2005, 'jan', 'Indiana', 'Total'): 2913.6,
             (2005, 'jan', 'Iowa', 'Total'): 1431.4,
             (2005, 'jan', 'Kansas', 'Total'): 1309.6,
             (2005, '

In [27]:

data = []
for (year, month, state, category), value in new_labor_df.items():
    data.append([year, month, state, category, value])

df = pd.DataFrame(data, columns=['Year', 'Month', 'State', 'Category', 'Value'])

# Print the cleaned DataFrame
print(df)
df.to_csv("./data/labor_processed/labor.csv")

       Year Month           State Category    Value
0      2005   jan         Alabama    Total   1904.9
1      2005   jan          Alaska    Total    288.6
2      2005   jan         Arizona    Total     2408
3      2005   jan        Arkansas    Total   1151.6
4      2005   jan      California    Total  14507.4
...     ...   ...             ...      ...      ...
42283  2020   oct   West Virginia      Gov    147.5
42284  2020   oct       Wisconsin      Gov    389.3
42285  2020   oct         Wyoming      Gov       68
42286  2020   oct     Puerto Rico      Gov    200.4
42287  2020   oct  Virgin Islands      Gov     10.9

[42288 rows x 5 columns]
